In [2]:
import os
import pandas as pd
import geopandas as gpd
from shapely.wkt import loads
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.font_manager
import seaborn as sn
from scipy.stats import zscore
import numpy as np
#matplotlib.font_manager.findSystemFonts(fontpaths=None, fontext='ttf')
sn.set_style("white")

from scipy.spatial import cKDTree

csfont = {'fontname':'Adobe Garamond Pro','fontsize':30}
hfont = {'fontname':'Adobe Garamond Pro','fontsize':12}

In [3]:
dd = pd.read_csv(r"G:\Mi unidad\walknet_datalake\extractions\pois_other_300.csv")

In [4]:
df = pd.read_csv(r"G:\Mi unidad\walknet_datalake\sources\edm2018\level1\level1_edm2018.csv",sep=",")
print(len(df))
df = df[df['ZT1259'].str.startswith(("005", "006", "007", "022", "026", "045", "049", "058", "065", "073", "074", "080", "084", "092", "104", "106", "113", "115", "127", "130", "134", "148", "167", "176", "177", "181", "184", "079","123","903"))]
df['CODMUNI'] = df['CODMUNI'].astype(str).str.zfill(3)
df['edge_id'] = df['VORIZT1259']+"|"+df['VDESZT1259']
relations = df['edge_id'].unique()
df.drop_duplicates(subset=['ID_HOGAR','ID_IND','ID_VIAJE'],inplace=True)
df = df[df['VDES']!='Casa']
df = df[df['VORI']=='Casa']
df = df[df['C8ACTIV']!='Otra situación']
df = df[df['CPMR'] == 'No']
#df = df[df['EMODO'].isin(['Andando/ pie','Coche pasajero','Coche conductor'])]
df = df[~df['VFRECUENCIA'].isin(['Alguna vez','Es la primera vez que hago el viaje'])]
df = df[df['MOTIVO_PRIORITARIO'].isin(['Compras','Estudio', 'Ocio', 'Deporte/ dar un paseo','Acompañamiento a otra persona','Médico'])]
df = df[df['DISTANCIA_VIAJE'] < 900]

266994


In [5]:
df['VORIHORAINI'] = df['VORIHORAINI'].astype(int).astype(str).str.zfill(4).str[:2] + ":" + df['VORIHORAINI'].astype(int).astype(str).str.zfill(4).str[2:]
df['VDESHORAFIN'] = df['VDESHORAFIN'].astype(int).astype(str).str.zfill(4).str[:2] + ":" + df['VDESHORAFIN'].astype(int).astype(str).str.zfill(4).str[2:]

In [6]:
df = df[~df['VDESHORAFIN'].str.startswith(("25","26"))]
df = df[~df['VORIHORAINI'].str.startswith(("25","26"))]

df.loc[df['VORIHORAINI'].str.startswith("24"),'VORIHORAINI'] = df[df['VORIHORAINI'].str.startswith("24")]['VORIHORAINI'].str.replace("24:","00:")
df.loc[df['VDESHORAFIN'].str.startswith("24"),'VDESHORAFIN'] = df[df['VDESHORAFIN'].str.startswith("24")]['VDESHORAFIN'].str.replace("24:","00:")

df['VDESHORAFIN'] = pd.to_datetime(df['VDESHORAFIN'], format= '%H:%M' )
df['VORIHORAINI'] = pd.to_datetime(df['VORIHORAINI'], format= '%H:%M' )

In [7]:
df['DURACION_VIAJE'] = ((df['VDESHORAFIN'] - df['VORIHORAINI']).dt.seconds)/60

df['VELOCIDAD_VIAJE'] = df['DISTANCIA_VIAJE']/(df['DURACION_VIAJE']/60)

df['DURACION_VIAJE'].fillna(0,inplace=True)
df['VELOCIDAD_VIAJE'].fillna(0,inplace=True)

C:\Users\katul\AppData\Local\Temp\ipykernel_4428\1053556119.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['DURACION_VIAJE'].fillna(0,inplace=True)
C:\Users\katul\AppData\Local\Temp\ipykernel_4428\1053556119.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, w

In [8]:
df['VDESHORAFIN'] = df['VDESHORAFIN'].dt.hour
df['VORIHORAINI'] = df['VORIHORAINI'].dt.hour

In [9]:
categorical = {
    'C2SEXO':('Gender',{'Hombre':'Male','Mujer':'Female'},(0,0)),
    'C4NAC':('Foreign',{'No':'No','Si':'Yes'},(0,1)),
    'CPMR':('PRM',{'No':'No','Si':'Yes'},(1,0)),
    'C13TARJETA':('TravelCard',{'Tarjeta de Transporte Público TTP':"Yes", 'Ninguna':"No", 'Ambas tarjetas':"Yes",'Tarjeta MULTI':"Yes"},(1,1)),
    'VVEHICULO':('HasVehicle',{'Si':'Yes','No':'No'},(2,0)),
    'C6CARNE':('HasLicense',{'No tiene':"No", 'Carné de coche (B) o superior  (C, D ó E)':"Yes",'Carné de moto y coche':"Yes", 'Carné de moto A1/A2':"Yes", 'Sólo licencia de ciclomotor':"Yes"},(2,1)),
    'VFRECUENCIA':('Frequency',{'A diario, de lunes a viernes':"Daily",'Entre 2 y 4 días laborables por semana':"Frequently",'Es la primera vez que hago el viaje':"0",'Menos de dos días laborables por semana':"Less Frequently", 'Alguna vez':"Sometimes"},(3,0)),
    'DIASEM':('DayOfWeek',{"Lunes":"Monday","Martes":"Tuesday","Miércoles":"Wednesday","Jueves":"Thursday"},(3,1)),
    'C7ESTUD':('EducationLevel',{'Primera etapa de Educación Secundaria y similar':"Second - A",'Educación Primaria':"First",'Segunda etapa de Educación Secundaria y similar':"Second - B",'Menos que Primaria':"No",'Enseñanzas de Formación Profesional de grado superior y equivalentes':"Third",'Educación postsecundaria no superior':"Second - B", 'Grados universitarios/licenciaturas/másteres y enseñanzas de doctorado':"Third"},(4,0)),
    'C8ACTIV':('Activity',{'Parado, ha trabajado antes':"Unemployed", 'Jubilado / Retirado/ Pensionista':"Retired",'Estudiante':"Student", 'Trabaja':"Worker", 'Trabaja y estudia':"Worker",'Cuidado de familiares':"Caretaker", 'Trabajo doméstico no remunerado':"Caretaker", 'Parado, busca primer trabajo':"Unemployed"},(4,1)),
    'VNOPRIVADO':("NoPrivate",{'No me gusta el coche':"Dislike", 'Más incómodo':"Less Convenient",'Evitar el atasco':"Congestion",'Contamino menos':"Other",'Dificultad de aparcamiento':"Hard to Park",'Otros':"Other",'Tardo más':"Takes Longer",'Es más caro':"Other"},(5,0)),
    'VNOPUBLICO':("NoPublic",{'No hay servicio público':"Bad Service",'Es más caro':"Other",'Tardo más':"Takes Longer",'El destino está muy cerca':"Very Close",'Otros':"Other",'Necesito mi vehículo para trabajar o gestión personal':"Other",'Mala combinación del tte.público':"Bad Service",'Prefiero ir andando / en bicicleta':"Prefer Active Modes",'Mi situación personal condiciona esta elección modal':"Bad Service",'Más incómodo':"Less Convenient", 'Por desconocimiento/falta de información':"Other",'No me gusta el tte. público':"Other"},(5,1)),
    "MOTIVO_PRIORITARIO":("TripPurpose",{'Compras':"Shopping",'Ocio':"Leisure",'Estudio':"Study",'Trabajo':"Work",'Deporte/ dar un paseo':"Sport / Stroll",'Otro domicilio':"Other",'Asunto personal':"Other",'Acompañamiento a otra persona':"Care",'Médico':"Care",'Gestión de trabajo':"Work",'Otros':"Other"},(6,0)),
    "MODO_PRIORITARIO":("TransportMode",{'Coche conductor particular' :"Motor - Personal",'Coche pasajero particular':"Motor - Personal",'Moto/ciclomotor particular':"Motor - Personal",'Andando/pie' : "Walking",'Autobus urbano Madrid EMT':"Bus",'Autobus interurbano':"Bus",'Autobus discrecional':"Bus",'Autobus de largo recorrido':"Bus",'Autobus urbano otro municipio':"Bus",'Otros':"Other",'Metro':"Train",'Renfe Cercanías':"Train",'Resto renfe':"Train",'Metro ligero/tranvía':"Train", 'Coche conductor empresa' : "Motor - Shared",'Coche pasajero empresa': "Motor - Shared",'Taxi':"Motor - Shared",'Coche pasajero alquiler con conductor': "Motor - Shared",'Coche conductor alquiler sin conductor': "Motor - Shared",'Moto/ciclomotor empresa': "Motor - Shared",'Moto/ciclomotor publica': "Motor - Shared",'Bicicleta particular':"Cycling",'Bicicleta publica':"Cycling",'Bicicleta empresa':"Cycling"},(6,1))
    }

for c,i in categorical.items():

    df[c] = df[c].map(i[1])
    df.rename(columns={c:i[0]},inplace=True)

In [10]:
df = df[df['VELOCIDAD_VIAJE'] < 80]

df = df[df['DURACION_VIAJE'] < 300]
df = df[df['DURACION_VIAJE'] >= 0]

df = df[df['N_VIAJES_POR_HOGAR'] < 20]

df = df[df['DISTANCIA_VIAJE'] < 20]


In [11]:
df.rename(columns = {'DISTANCIA_VIAJE':'TripDistance'},inplace=True)

In [12]:
continuous = {
    'EDAD_FIN':('Age',(0,0)),
    'N_MIEMBROS_POR_HOGAR':('HouseholdSize',(0,1)),
    'N_VIAJES_POR_HOGAR':('TripsHousehold',(1,0)),
    'N_ETAPAS_POR_VIAJE':('TripStages',(1,1)),
    'VORIHORAINI':('StartTime',(2,0)),
    'VDESHORAFIN':('EndTime',(2,1)),
    'DURACION_VIAJE':('TripDuration',(3,0)),
    'VELOCIDAD_VIAJE':('TripSpeed',(3,1))
    }

for c,i in continuous.items():

    df.rename(columns={c:i[0]},inplace=True)
    df[i[0]] = df[i[0]].astype(float)



In [13]:
df.loc[df['Age']<5,'Age Status'] = "Less than 5"
df.loc[(df['Age']>=5) & (df['Age']<12),'Age Status'] = "Ages 6-12"
df.loc[(df['Age']>=12) & (df['Age']<18),'Age Status'] = "Ages 13-18"
df.loc[(df['Age']>=18) & (df['Age']<40),'Age Status'] = "Ages 19-40"
df.loc[(df['Age']>=40) & (df['Age']<65),'Age Status'] = "Ages 41-65"
df.loc[df['Age']>=65,'Age Status'] = "More than 65"

In [14]:

hd = {}
h15 = {}
h30 = {}
for h in list(df['ID_HOGAR'].unique()):
    dist = list(df[df['ID_HOGAR'] == h]['TripDuration'].values)
    
    if all(di < 30 for di in dist): h30.update({h:1})
    else: h30.update({h:0})
    
    if all(di < 15 for di in dist): h15.update({h:1})
    else: h15.update({h:0})

    hd.update({h:'Other'})
    d = df[df['ID_HOGAR'] == h][['ID_IND','Age','Age Status','HouseholdSize','Activity']].drop_duplicates(subset='ID_IND')
    if d['HouseholdSize'].values[0] == 1:
        if d['Activity'].values[0] == 'Retired': hd.update({h:'Retirees'})
        elif d['Activity'].values[0] == 'Student': hd.update({h:'Students (age 19+)'})
        elif d['Activity'].values[0] in ['Worker']: hd.update({h:'Full-Time Workers'})
        else: hd.update({h:'Other'})
    else:
        if any(act in ['Retired'] for act in d['Activity'].values): hd.update({h:'Retirees'})
        if any(act in ['Worker'] for act in d['Activity'].values): hd.update({h:'Full-Time Workers'})
        if any(age == "Ages 13-18" for age in d['Age Status'].values): hd.update({h:'Students (Age 13–18)'})
        if any(age == "Ages 6-12" for age in d['Age Status'].values): hd.update({h:'Students (Age 6–12)'})
        if any(age == "Less than 5" for age in d['Age Status'].values): hd.update({h:'Children'})
        
df['HouseholdStructure'] = df['ID_HOGAR'].map(hd)



In [15]:
df['Trip15'] = ""
df.loc[df['TripDuration']<=15,'Trip15'] = 'Yes'
df.loc[df['TripDuration']>15,'Trip15'] = 'No'

In [16]:
df.drop(columns=['ID_ETAPA','Foreign', 'C5CAM','C9PROF', 'C10SECTOR','C14ABONO', 'DDIA','DMES', 'DANNO', 'DayOfWeek', 'DNOVIAJO','C11ZT1259', 'C12ZT1259','PRM', 'TIPO_ENCUESTA','NOMMUNI', 'CODPROV', 'NOMPROV',
       'ZT1259', 'CZ208', 'ELE_HOGAR_NUEVO', 'A1PER', 'A2PER4', 'B1NVE',
       'V1B11TIPO', 'V1B12CARB', 'V1B13EST', 'V2B11TIPO1', 'V2B12CARB1',
       'V2B13EST1', 'V3B11TIPO1', 'V3B12CARB1', 'V3B13EST1', 'V4B11TIPO1',
       'V4B12CARB1', 'V4B13EST1', 'V5B11TIPO1', 'V5B12CARB1', 'V5B13EST1','EMODO', 'EMODO1','MODO', 'ELINEAEMPRESA_ORIGINAL', 'ESUBIDA', 'ESUBIDA_cod', 'EBAJADA',
       'EBAJADA_cod', 'ETITULO', 'EESTACIONA', 'EOCUPACION', 'ETDESPH',
       'COD_MUN_PARADA', 'ELE_G_POND_Esc2', 'VORI','VDES',
       'EndTime', 'Frequency',  'VDESZT1259', 'ELE_G_POND_ESC2', 'edge_id', 'TripSpeed','Trip15'],inplace=True)

In [17]:
map = {'CODMUNI':'id_municipality', 'VORIZT1259':'id_taz','ELE_G_POND':'elevator',  'ID_HOGAR':'id_household','ID_IND':'id_person', 'ID_VIAJE':'id_trip',
   'TripPurpose':'trip_purpose', 'TripDistance':'trip_distance','TripStages':'trip_stages', 'TripDuration':'trip_duration', 'TransportMode':'trip_mode',
   'Gender':'dem_gender', 'Age':'dem_age','Age Status':'dem_cohort','EducationLevel':'dem_education', 'Activity':'dem_activity',
   'HouseholdSize':'dem_hou_size','HouseholdStructure':'dem_hou_structure','TripsHousehold':'dem_hou_trips',
   'HasLicense':'dem_att_license', 'HasVehicle':'dem_att_vehicle',
   'TravelCard':'dem_att_card', 'NoPrivate':'dem_att_noprivate', 'NoPublic':'dem_att_nopublic'}

In [18]:
df = df[map.keys()].rename(columns=map)

In [19]:
df['id'] = df['id_household'].astype(int).astype(str)+df['id_person'].astype(int).astype(str)+df['id_trip'].astype(int).astype(str)

In [20]:
datasets = [
r"G:\Mi unidad\walknet_datalake\extractions\pois_network_centrality_300.csv",
r"G:\Mi unidad\walknet_datalake\extractions\pois_network_centrality_600.csv",
r"G:\Mi unidad\walknet_datalake\extractions\pois_network_centrality_900.csv",
r"G:\Mi unidad\walknet_datalake\extractions\pois_network_centrality_1200.csv",
r"G:\Mi unidad\walknet_datalake\extractions\pois_network_centrality_1500.csv",
r"G:\Mi unidad\walknet_datalake\extractions\pois_design_300.csv",
r"G:\Mi unidad\walknet_datalake\extractions\pois_design_600.csv",
r"G:\Mi unidad\walknet_datalake\extractions\pois_design_900.csv",
r"G:\Mi unidad\walknet_datalake\extractions\pois_design_1200.csv",
r"G:\Mi unidad\walknet_datalake\extractions\pois_design_1500.csv",
r"G:\Mi unidad\walknet_datalake\extractions\pois_other_300.csv",
r"G:\Mi unidad\walknet_datalake\extractions\pois_other_600.csv",
r"G:\Mi unidad\walknet_datalake\extractions\pois_other_900.csv",
r"G:\Mi unidad\walknet_datalake\extractions\pois_other_1200.csv",
r"G:\Mi unidad\walknet_datalake\extractions\pois_other_1500.csv",
r"G:\Mi unidad\walknet_datalake\extractions\pois_walkable_trips_300.csv",
r"G:\Mi unidad\walknet_datalake\extractions\pois_walkable_trips_600.csv",
r"G:\Mi unidad\walknet_datalake\extractions\pois_walkable_trips_900.csv",
r"G:\Mi unidad\walknet_datalake\extractions\pois_walkable_trips_1200.csv",
r"G:\Mi unidad\walknet_datalake\extractions\pois_walkable_trips_1500.csv",
r"G:\Mi unidad\walknet_datalake\extractions\pois_transacc_300.csv",
r"G:\Mi unidad\walknet_datalake\extractions\pois_transacc_600.csv",
r"G:\Mi unidad\walknet_datalake\extractions\pois_transacc_900.csv",
r"G:\Mi unidad\walknet_datalake\extractions\pois_transacc_1200.csv",
r"G:\Mi unidad\walknet_datalake\extractions\pois_transacc_1500.csv",
r"G:\Mi unidad\walknet_datalake\extractions\pois_parkacc_300.csv",
r"G:\Mi unidad\walknet_datalake\extractions\pois_parkacc_600.csv",
r"G:\Mi unidad\walknet_datalake\extractions\pois_parkacc_900.csv",
r"G:\Mi unidad\walknet_datalake\extractions\pois_parkacc_1200.csv",
r"G:\Mi unidad\walknet_datalake\extractions\pois_parkacc_1500.csv",
r"G:\Mi unidad\walknet_datalake\extractions\pois_far_localacc_300.csv",
r"G:\Mi unidad\walknet_datalake\extractions\pois_far_localacc_600.csv",
r"G:\Mi unidad\walknet_datalake\extractions\pois_far_localacc_900.csv",
r"G:\Mi unidad\walknet_datalake\extractions\pois_far_localacc_1200.csv",
r"G:\Mi unidad\walknet_datalake\extractions\pois_far_localacc_1500.csv"]


In [21]:
data = {d.split("\\")[-1].split(".")[0]:pd.read_csv(d) for d in datasets}

In [22]:
data['pois_network_centrality_900']

,tz_id,poi_id,des_mean_degree,des_straightness,geometry
0,079-09-204,28900-1-6659,2.665272,0.7796,0101000000A8C64BB755881A4177BE9FAAFD145141
1,079-09-204,28900-1-6661,2.665272,0.7796,0101000000B29DEF2755881A4183C0CA5DF7145141
2,079-09-204,28900-1-6680,2.665272,0.7796,01010000006ABC741368881A411D5A6417FB145141
3,079-09-204,28900-1-6660,2.665272,0.7796,0101000000A8C64BB755881A41FED47895FB145141
4,079-09-204,28900-1-6677,2.665272,0.7796,01010000002DB29D6F65881A41DF4F8D13FE145141
...,...,...,...,...,...
309072,176-003,28176-1-5574,NaN,NaN,01010000007368916D47A719419CC42078EF165141
309073,006-009,28006-1-1593,NaN,NaN,01010000000C022B07522E1B41A69BC4C4F11E5141
309074,006-009,28006-1-1608,NaN,NaN,0101000000E3A59B44692E1B419CC42044F01E5141
309075,079-16-443,28900-1-89070,NaN,NaN,010100000023DBF93EDE1A1B41AE47E152BC175141


In [106]:
for k,v in data.items():
    data[k] = v[['poi_id']+[c for c in v.columns if not c in ['poi_id','tz_id','local_population','geometry']]]
    #.set_index('poi_id')
    

In [107]:
for k,v in data.items():
    if 'pois_parkacc' in k:
        data[k] = pd.concat([v[v['park_type'] == t].set_index('poi_id')['unique_parks_count'].rename(f'acc_parks_{t}') for t in ["S","M","L"]],axis=1)
    else: data[k] = v.set_index('poi_id')


In [ ]:
data['pois_other_300'][data['pois_other_300']['des_building_age'].isnull()]

In [109]:
template = pd.read_csv(r"G:\Mi unidad\walknet_datalake\extractions\fn_pois_template.csv").set_index('poi_id')

In [110]:
template['geometry'] = template['geometry'].apply(loads)
template = gpd.GeoDataFrame(template, geometry='geometry', crs= 25830)

In [ ]:
template

In [112]:
cols = ['tz_id', 'node_id', 'local_population', 

'dem_income', 'dem_household_size', 'dem_mean_age',
'dem_population_18', 'dem_population_65',

'acc_care_other','acc_care_public',
'acc_school_superior', 'acc_school_basic',
'acc_leisure_bar', 'acc_leisure_cultural', 'acc_leisure_shows',
'acc_shopping_mall', 'acc_shopping_market', 'acc_shopping_alone',
'acc_sport_other',
'acc_transportation',
'acc_parks_S', 'acc_parks_M', 'acc_parks_L',
'dens_pop_total', 'dens_hou_total',
'dens_far', 'dens_far_ag',
'den_perc_unbuilt', 'dens_built_total',
'den_perc_housing_sfr','den_perc_housing_ch',
'den_perc_care_other', 'den_perc_care_public',
'den_perc_school_superior', 'den_perc_school_basic',
'den_perc_leisure_bar', 'den_perc_leisure_cultural',
'den_perc_leisure_shows', 'den_perc_shopping_mall',
'den_perc_shopping_market', 'den_perc_shopping_alone',
'den_perc_sport_other', 'den_perc_office', 'den_perc_industrial',
'den_perc_storage', 'den_perc_parking', 'den_perc_hotel',
'den_perc_religious', 'den_perc_infra', 

'div_wt_care', 'div_ut_care', 'div_wt_school', 'div_ut_school',
'div_wt_leisure', 'div_ut_leisure', 'div_wt_shopping',
'div_ut_shopping', 'div_wt_sport', 'div_ut_sport', 

'des_mean_degree', 'des_straightness', #'des_betweenness',
'des_total_length','des_block_length', 'des_culdesac', 'des_slope','des_building_age',
       
'geometry']

In [113]:
btw = gpd.read_file(r"G:\Mi unidad\walknet_datalake\temp\betweenness.gpkg")[['degree','closeness','betweenness','eigenvector','geometry']].rename(columns ={'degree':'des_degree','closeness':'des_closeness','betweenness':'des_betweenness','eigenvector':'des_eigenvector'})

In [114]:
df_300 = pd.merge(
    template,
    pd.concat([v for k,v in data.items() if "300" in k],axis=1),
    left_index=True,right_index=True, how='left')[cols]
df_600 = pd.merge(
    template,
    pd.concat([v for k,v in data.items() if "600" in k],axis=1),
    left_index=True,right_index=True, how='left')[cols]
df_900 = pd.merge(
    template,
    pd.concat([v for k,v in data.items() if "900" in k],axis=1),
    left_index=True,right_index=True, how='left')[cols]
df_1200 = pd.merge(
    template,
    pd.concat([v for k,v in data.items() if "1200" in k],axis=1),
    left_index=True,right_index=True, how='left')[cols]
df_1500 = pd.merge(
    template,
    pd.concat([v for k,v in data.items() if "1500" in k],axis=1),
    left_index=True,right_index=True, how='left')[cols]

In [115]:
df_300 = df_300[df_300['local_population'] > 0]
df_600 = df_600[df_600['local_population'] > 0]
df_900 = df_900[df_900['local_population'] > 0]
df_1200 = df_1200[df_1200['local_population'] > 0]
df_1500 = df_1500[df_1500['local_population'] > 0]

df_300 = df_300[df_300['des_building_age'] > 1880]
df_600 = df_600[df_600['des_building_age'] > 1880]
df_900 = df_900[df_900['des_building_age'] > 1880]
df_1200 = df_1200[df_1200['des_building_age'] > 1880]
df_1500 = df_1500[df_1500['des_building_age'] > 1880]

In [116]:
def join_by_nearest(gdf1,gdf2):
    # Suponiendo que gdf1 y gdf2 son tus GeoDataFrames de puntos
    gdf1_coords = gdf1.geometry.apply(lambda geom: (geom.x, geom.y)).tolist()
    gdf2_coords = gdf2.geometry.apply(lambda geom: (geom.x, geom.y)).tolist()

    # Crear árboles para ambas capas
    tree_gdf1 = cKDTree(gdf1_coords)
    tree_gdf2 = cKDTree(gdf2_coords)

    # Buscar los puntos más cercanos en gdf2 para cada punto en gdf1
    distances, indices = tree_gdf1.query(gdf2_coords, k=1)

    # Filtrar índices que estén fuera de los límites
    valid_indices = [i for i in range(len(indices)) if indices[i] < len(gdf1)]

    # Crear un DataFrame con resultados válidos
    results = pd.DataFrame({
        'gdf1_index': [indices[i] for i in valid_indices],
        'gdf2_index': valid_indices,
        'distance': [distances[i] for i in valid_indices]
    })

    # Verificar que no haya duplicados en la relación 1 a 1
    duplicated_gdf1 = results.duplicated(subset='gdf1_index', keep=False)
    duplicated_gdf2 = results.duplicated(subset='gdf2_index', keep=False)

    if not duplicated_gdf1.any() and not duplicated_gdf2.any():
        print("Relación 1 a 1 verificada")
    else:
        print("Se encontraron duplicados, la relación no es 1 a 1")

    # Unir los GeoDataFrames usando los índices válidos obtenidos
    gdf1_matched = gdf1.iloc[results['gdf1_index']].reset_index(drop=True)
    gdf2_matched = gdf2.iloc[results['gdf2_index']].reset_index(drop=True)

    # Resultado unido
    return gdf1_matched.join(gdf2_matched, lsuffix='', rsuffix='_gdf2')


In [ ]:
df_300 = join_by_nearest(df_300,btw)#.drop('geometry_gdf2')
df_600 = join_by_nearest(df_600,btw)
df_900 = join_by_nearest(df_900,btw)
df_1200 = join_by_nearest(df_1200,btw)
df_1500 = join_by_nearest(df_1500,btw)

In [118]:
df_300 = df_300.set_crs(25830)
df_600 = df_600.set_crs(25830)
df_900 = df_900.set_crs(25830)
df_1200 = df_1200.set_crs(25830)
df_1500 = df_1500.set_crs(25830)

In [119]:
#df_900.drop(columns=['geometry_gdf2']).to_file(r"G:\Mi unidad\walknet_datalake\temp\df_900.gpkg",driver='GPKG')

In [120]:
def agg_tzs(df):
    df = df[df['local_population']!=0]
    # Excluir las columnas no deseadas al definir las columnas para el promedio ponderado
    columns_to_average = [col for col in df.columns if col not in ['geometry', 'node_id', 'tz_id', 'local_population', 'geometry_gdf2', 'hex_id', 'id', 'left', 'top', 'right', 'bottom', 'row_index', 'col_index']]
    # Función para calcular el promedio ponderado
    def weighted_average(dfg):
        weights = dfg['local_population']
        return pd.Series({
            col: (dfg[col] * weights).sum() / weights.sum() for col in columns_to_average
        })
    
    # Agrupar por el ID de hexágono y calcular el promedio ponderado para cada columna
    df_aggregated = df.groupby('tz_id')[['local_population']+columns_to_average].apply(weighted_average).reset_index()
    
    return df_aggregated


In [121]:

def agg_tzs(df):
    df = df[df['local_population'] != 0]
    # Exclude unwanted columns from the weighted average calculation
    columns_to_average = [
        col for col in df.columns if col not in [
            'geometry', 'node_id', 'tz_id', 'local_population', 
            'geometry_gdf2', 'hex_id', 'id', 'left', 'top', 
            'right', 'bottom', 'row_index', 'col_index'
        ]
    ]
    
    # Function to calculate the weighted average for each column
    def weighted_average(dfg):
        weights = dfg['local_population']
        result = {}
        for col in columns_to_average:
            # Filter out rows with NaN values in the current column
            valid_rows = dfg[~dfg[col].isna()]
            if valid_rows.empty:
                result[col] = None  # Assign None if no valid rows are left
            else:
                weights_filtered = valid_rows['local_population']
                result[col] = (valid_rows[col] * weights_filtered).sum() / weights_filtered.sum()
        return pd.Series(result)
    
    # Group by tz_id and calculate the weighted average for each column
    df_aggregated = df.groupby('tz_id')[['local_population'] + columns_to_average].apply(weighted_average).reset_index()
    
    return df_aggregated



In [122]:
df_tz_300 = agg_tzs(df_300)
df_tz_600 = agg_tzs(df_600)
df_tz_900 = agg_tzs(df_900)
df_tz_1200 = agg_tzs(df_1200)
df_tz_1500 = agg_tzs(df_1500)

In [ ]:
df_tz_900[df_tz_900['des_building_age'].isna()]

In [ ]:
df_tz_300[df_tz_300['des_building_age'].isna()]

In [125]:
df_tz_1500['des_building_age'] = df_tz_1500['des_building_age'].fillna(0).astype(int)
df_tz_1200['des_building_age'] = df_tz_1200['des_building_age'].fillna(0).astype(int)
df_tz_900['des_building_age'] = df_tz_900['des_building_age'].fillna(0).astype(int)
df_tz_600['des_building_age'] = df_tz_600['des_building_age'].fillna(0).astype(int)
df_tz_300['des_building_age'] = df_tz_300['des_building_age'].fillna(0).astype(int)


In [ ]:
df_tz_900['des_building_age'].unique()

In [127]:
bins = [1880, 1930, 1960, 1980, 2000, 2010, 2018]
labels = ['1880-1930', '1930-1960', '1960-1980', '1980-2000', '2000-2010', '2010-2018']

# Use pd.cut to bucketize
df_tz_300['des_age_range'] = pd.cut(df_tz_300['des_building_age'], bins=bins, labels=labels, right=False)
df_tz_600['des_age_range'] = pd.cut(df_tz_600['des_building_age'], bins=bins, labels=labels, right=False)
df_tz_900['des_age_range'] = pd.cut(df_tz_900['des_building_age'], bins=bins, labels=labels, right=False)
df_tz_1200['des_age_range'] = pd.cut(df_tz_1200['des_building_age'], bins=bins, labels=labels, right=False)
df_tz_1500['des_age_range'] = pd.cut(df_tz_1500['des_building_age'], bins=bins, labels=labels, right=False)

In [128]:
labels = ['den_hou_Q1','den_hou_Q2','den_hou_Q3','den_hou_Q4']

# Use pd.cut to bucketize
df_tz_300['den_quartile'] = pd.qcut(df_tz_300['dens_hou_total'], 4, labels=labels)
df_tz_600['den_quartile'] = pd.qcut(df_tz_600['dens_hou_total'], 4, labels=labels)
df_tz_900['den_quartile'] = pd.qcut(df_tz_900['dens_hou_total'], 4, labels=labels)
df_tz_1200['den_quartile'] = pd.qcut(df_tz_1200['dens_hou_total'], 4, labels=labels)
df_tz_1500['den_quartile'] = pd.qcut(df_tz_1500['dens_hou_total'], 4, labels=labels)

In [129]:
ds_1500 = pd.merge(df, df_tz_1500, left_on='id_taz',right_on="tz_id",how='left')
ds_1200 = pd.merge(df, df_tz_1200, left_on='id_taz',right_on="tz_id",how='left')
ds_900 = pd.merge(df, df_tz_900, left_on='id_taz',right_on="tz_id",how='left')
ds_600 = pd.merge(df, df_tz_600, left_on='id_taz',right_on="tz_id",how='left')
ds_300 = pd.merge(df, df_tz_300, left_on='id_taz',right_on="tz_id",how='left')

In [130]:
df_testlogit = ds_1500[['elevator','trip_purpose', 'trip_distance', 'trip_mode', 'dem_gender', 'dem_income','dem_age','dem_education', 'dem_activity', 'dem_hou_size','acc_shopping_market', 'acc_shopping_alone']]
df_testlogit = df_testlogit[df_testlogit['dem_income'].notnull()]
for v in ['acc_shopping_market', 'acc_shopping_alone']:
    df_testlogit[v] = df_testlogit[v].fillna(0)
df_testlogit['acc_shopping'] = df_testlogit['acc_shopping_alone'] + df_testlogit['acc_shopping_market']
df_testlogit = df_testlogit.drop(columns = ['acc_shopping_market', 'acc_shopping_alone'])
df_testlogit = df_testlogit[df_testlogit['trip_purpose'] == 'Shopping']
df_testlogit['dem_education'] = df_testlogit['dem_education'].map({'Second - B':3, 'First':1, 'Third':4, 'Second - A':2, 'No':0})
df_testlogit['dem_activity'] = df_testlogit['dem_activity'].map({'Retired':0, 'Caretaker':1, 'Unemployed':0, 'Worker':1, 'Student':1})
df_testlogit['dem_gender'] = df_testlogit['dem_gender'].map({'Female':0,'Male':1})
df_testlogit = df_testlogit[df_testlogit['trip_mode'] != 'Other']
df_testlogit['trip_mode'] = df_testlogit['trip_mode'].map({'Walking':'active', 'Motor - Personal':'driving', 'Bus':'transit', 'Cycling':'active', 'Train':'transit','Motor - Shared':'driving'})
df_testlogit = df_testlogit[(df_testlogit['dem_age']>=18)&(df_testlogit['dem_age']<=65)]
df_testlogit = df_testlogit[df_testlogit['trip_mode'].isin(['active','driving'])]

In [ ]:
df_testlogit.columns

In [132]:
df_testlogit.to_csv(r"C:\Users\katul\OneDrive\Escritorio\df_testlogit.csv",index=None)

In [174]:
from sklearn.utils import resample
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from sklearn.metrics import roc_curve, roc_auc_score, accuracy_score
import numpy as np


In [187]:
csfont = {'fontname':'Adobe Garamond Pro','fontsize':30}
hfont = {'fontname':'Adobe Garamond Pro','fontsize':15}

In [ ]:
# Step 1: Filter dataset for valid rows
data_filtered = df_testlogit[(df_testlogit['dem_age'] >= 18) & (df_testlogit['dem_age'] <= 65)]
# Step 2: Balance the `trip_mode` column
active = data_filtered[data_filtered['trip_mode'] == 'active']
driving = data_filtered[data_filtered['trip_mode'] == 'driving']
# Resample the minority class to balance the dataset
min_class_count = min(len(active), len(driving))
active_resampled = resample(active, replace=False, n_samples=min_class_count, random_state=42)
driving_resampled = resample(driving, replace=False, n_samples=min_class_count, random_state=42)
# Combine the balanced dataset
data_filtered = pd.concat([active_resampled, driving_resampled])

# Step 1: Define features and target variable
features = ['trip_distance', 'dem_gender', 'dem_income', 'dem_age','dem_education', 'dem_activity', 'dem_hou_size', 'acc_shopping']
target = 'trip_mode'
# Encode the target variable: 'active' -> 0, 'driving' -> 1
data_filtered['trip_mode_encoded'] = data_filtered['trip_mode'].map({'active': 0, 'driving': 1})
# Step 2: Scale numeric features
scaler = StandardScaler()
data_filtered[features] = scaler.fit_transform(data_filtered[features])
# Step 3: Split into training and testing sets
X = data_filtered[features]
y = data_filtered['trip_mode_encoded']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

# Display the shape of the training and testing sets
X_train.shape, X_test.shape, y_train.shape, y_test.shape

# Add a constant term for the intercept in the model
X_train_const = sm.add_constant(X_train)

# Fit the Binomial Logit Model using the training data and weights
logit_model = sm.Logit(y_train, X_train_const)
result = logit_model.fit()

# Display the summary of the model
result.summary()

# Prepare test data with a constant term
X_test_const = sm.add_constant(X_test)

# Predict probabilities for the test set
y_pred_prob = result.predict(X_test_const)

# Convert probabilities to binary predictions using 0.5 as threshold
y_pred = (y_pred_prob >= 0.5).astype(int)

# Evaluate model performance
accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_prob)

# Plot ROC Curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'ROC Curve (AUC = {roc_auc:.2f})',color='r')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray', label='Random Guess')
plt.xlabel('False Positive Rate',**hfont)
plt.ylabel('True Positive Rate',**hfont)
plt.title('ROC Curve No Distance Threshold',**csfont)
plt.legend()
plt.grid()
plt.tight_layout()
plt.savefig(r"C:\Users\katul\OneDrive\Escritorio\ROC_Binary Logit Model.jpg")
plt.show()

accuracy, roc_auc

# Extract coefficients and their confidence intervals
coefficients = result.params
conf_intervals = result.conf_int()
features_with_const = ['Intercept'] + features  # Include the constant term

# Create a DataFrame for visualization
impact_df = pd.DataFrame({
    'Feature': features_with_const,
    'Coefficient': coefficients.values,
    'Lower CI': conf_intervals[0].values,
    'Upper CI': conf_intervals[1].values
}).set_index('Feature')

# Sort by absolute coefficient values for better visualization
impact_df['Abs Coefficient'] = np.abs(impact_df['Coefficient'])
impact_df = impact_df.sort_values(by='Abs Coefficient', ascending=False)

# Plot feature impacts with confidence intervals
plt.figure(figsize=(8.5, 6))
plt.errorbar(
    impact_df.index, impact_df['Coefficient'], 
    yerr=(impact_df['Coefficient'] - impact_df['Lower CI'], impact_df['Upper CI'] - impact_df['Coefficient']),
    fmt='o', capsize=5, label="Coefficient ± CI",color='r'
)
plt.axhline(0, color='gray', linestyle='--', linewidth=0.8)
plt.xticks(rotation=45)
plt.title('Feature Impact on Mode Choice\nNo Distance Threshold',**csfont)
plt.ylabel('Coefficient',**hfont)
plt.xlabel('Feature',**hfont)
plt.tight_layout()
plt.grid()
plt.legend()
plt.tight_layout()
plt.savefig(r"C:\Users\katul\OneDrive\Escritorio\FeatureImpact Logit Model.jpg")
plt.show()

import statsmodels.api as sm

# Add a constant term for the intercept
X_train_const = sm.add_constant(X_train)

# Fit the Binomial Logit Model
logit_model = sm.Logit(y_train, X_train_const)
result = logit_model.fit()

# Summary and ROC Curve
print(result.summary())
y_pred_prob = result.predict(sm.add_constant(X_test))
#plot_roc_curve(y_test, y_pred_prob)


In [ ]:
#FILTER AGAIN FOR DISTANCE IN PROXIMITY

# Step 1: Filter dataset for valid rows
data_filtered = df_testlogit[(df_testlogit['dem_age'] >= 18) & (df_testlogit['dem_age'] <= 65)]
data_filtered = data_filtered[data_filtered['trip_distance']<1.5]
# Step 2: Balance the `trip_mode` column
active = data_filtered[data_filtered['trip_mode'] == 'active']
driving = data_filtered[data_filtered['trip_mode'] == 'driving']
# Resample the minority class to balance the dataset
min_class_count = min(len(active), len(driving))
active_resampled = resample(active, replace=False, n_samples=min_class_count, random_state=42)
driving_resampled = resample(driving, replace=False, n_samples=min_class_count, random_state=42)
# Combine the balanced dataset
data_filtered = pd.concat([active_resampled, driving_resampled])

# Step 1: Define features and target variable
features = ['trip_distance', 'dem_gender', 'dem_income', 'dem_age','dem_education', 'dem_activity', 'dem_hou_size', 'acc_shopping']
target = 'trip_mode'
# Encode the target variable: 'active' -> 0, 'driving' -> 1
data_filtered['trip_mode_encoded'] = data_filtered['trip_mode'].map({'active': 0, 'driving': 1})
# Step 2: Scale numeric features
scaler = StandardScaler()
data_filtered[features] = scaler.fit_transform(data_filtered[features])
# Step 3: Split into training and testing sets
X = data_filtered[features]
y = data_filtered['trip_mode_encoded']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

# Display the shape of the training and testing sets
X_train.shape, X_test.shape, y_train.shape, y_test.shape

# Add a constant term for the intercept in the model
X_train_const = sm.add_constant(X_train)

# Fit the Binomial Logit Model using the training data and weights
logit_model = sm.Logit(y_train, X_train_const)
result = logit_model.fit()

# Display the summary of the model
result.summary()

# Prepare test data with a constant term
X_test_const = sm.add_constant(X_test)

# Predict probabilities for the test set
y_pred_prob = result.predict(X_test_const)

# Convert probabilities to binary predictions using 0.5 as threshold
y_pred = (y_pred_prob >= 0.5).astype(int)

# Evaluate model performance
accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_prob)

# Plot ROC Curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'ROC Curve (AUC = {roc_auc:.2f})',color='r')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray', label='Random Guess')
plt.xlabel('False Positive Rate',**hfont)
plt.ylabel('True Positive Rate',**hfont)
plt.title('ROC Curve 1500 m',**csfont)
plt.legend()
plt.grid()
plt.tight_layout()
plt.savefig(r"C:\Users\katul\OneDrive\Escritorio\ROC_Binary Logit Model Proximity.jpg")
plt.show()

print(accuracy, roc_auc)

# Extract coefficients and their confidence intervals
coefficients = result.params
conf_intervals = result.conf_int()
features_with_const = ['Intercept'] + features  # Include the constant term

# Create a DataFrame for visualization
impact_df = pd.DataFrame({
    'Feature': features_with_const,
    'Coefficient': coefficients.values,
    'Lower CI': conf_intervals[0].values,
    'Upper CI': conf_intervals[1].values
}).set_index('Feature')

# Sort by absolute coefficient values for better visualization
impact_df['Abs Coefficient'] = np.abs(impact_df['Coefficient'])
impact_df = impact_df.sort_values(by='Abs Coefficient', ascending=False)

# Plot feature impacts with confidence intervals
plt.figure(figsize=(8.5, 6))
plt.errorbar(
    impact_df.index, impact_df['Coefficient'], 
    yerr=(impact_df['Coefficient'] - impact_df['Lower CI'], impact_df['Upper CI'] - impact_df['Coefficient']),
    fmt='o', capsize=5, label="Coefficient ± CI",color='r'
)
plt.axhline(0, color='gray', linestyle='--', linewidth=0.8)
plt.xticks(rotation=45)
plt.title('Feature Impact on Mode Choice\n1500 m Threshold',**csfont)
plt.ylabel('Coefficient',**hfont)
plt.xlabel('Feature',**hfont)
plt.tight_layout()
plt.grid()
plt.legend()
plt.tight_layout()
plt.savefig(r"C:\Users\katul\OneDrive\Escritorio\FeatureImpact Logit Model Proximity.jpg")
plt.show()

import statsmodels.api as sm

# Add a constant term for the intercept
X_train_const = sm.add_constant(X_train)

# Fit the Binomial Logit Model
logit_model = sm.Logit(y_train, X_train_const)
result = logit_model.fit()

# Summary and ROC Curve
print(result.summary())
y_pred_prob = result.predict(sm.add_constant(X_test))
#plot_roc_curve(y_test, y_pred_prob)


In [48]:
from itertools import product

lists = {
        'trip_purpose' : ds_900.trip_purpose.unique(),
        'dem_cohort' : ds_900.dem_cohort.unique()}

lists_with_all = {key: list(values) + ["all"] for key, values in lists.items()}

# Generate all combinations of the lists
combinations = product(*lists_with_all.values())

# Build dictionaries for each combination
dict_combinations = [dict(zip(lists_with_all.keys(), combination)) for combination in combinations]


In [49]:
control_combinations  = {"|".join([str(e) for e in di.values()]):di for di in dict_combinations}

In [50]:
# Function to filter DataFrame based on a dictionary
def filter_dataframe(df, filter_dict):
    filtered_df = df.copy()
    for key, value in filter_dict.items():
        if value != "all":
            filtered_df = filtered_df[filtered_df[key] == value]
    return filtered_df

In [51]:
# Create one DataFrame per combination
target_dataframes = {'300':{},'600':{},'1200':{},'900':{},'1500':{}}
for dft in ('300',ds_300),('600',ds_600),('900',ds_900),('1200',ds_1200),('1500',ds_1500):
    for name, combo in control_combinations.items():
        filtered_df = filter_dataframe(dft[1], combo)
        target_dataframes[dft[0]].update({name: filtered_df})

In [ ]:
# Display the filtered DataFrames (or use them as needed)
i = 0
for dist, dat in target_dataframes.items():
    for name, filtered_df in dat.items():
        if not filtered_df.empty:
            i+= 1
            print(f"Distance threshold {dist}")
            print(f"Combination: {name}")
            print(str(len(filtered_df)))
            print(str(filtered_df['elevator'].sum()))
            print("-" * 40)
        

In [53]:
sample_metrics = {}
for dist, dfs in target_dataframes.items():
    for name, df in dfs.items():
        if df['elevator'].sum() > 45000:
            table_size = len(df)
            sample_size = df['elevator'].sum()
            sample_walk = df[df['trip_mode']=='Walking']['elevator'].sum()
            sample_drive = df[df['trip_mode']=='Motor - Personal']['elevator'].sum()
            walk_perc = round((100 * sample_walk) / sample_size,0)
            drive_perc = round((100 * sample_drive) / sample_size,2)
            sample_metrics.update({f"{dist}|{name}":{'table_size': table_size,'sample_size': sample_size,'sample_walk': sample_walk,'sample_drive': sample_drive, 'walk_perc':walk_perc, 'drive_perc':drive_perc}})



In [54]:
sample_metrics = pd.DataFrame.from_dict(sample_metrics, orient='index').fillna(0)
sample_metrics['sample_size'] = sample_metrics.sample_size.astype(int)

In [55]:
sample_metrics.reset_index(inplace=True)
sample_metrics = sample_metrics[sample_metrics['index'].str.contains('1500')]

In [56]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


# Sort the data by sample_size in descending order

df_sorted = sample_metrics.copy()
df_sorted.set_index('index',inplace=True)

df_sorted['walk_size'] = df_sorted['sample_walk'] 
df_sorted['drive_size'] = df_sorted['sample_drive']

# Ensure proper sorting and data handling
df_sorted = df_sorted.sort_values(by='sample_size', ascending=True)





In [ ]:
fig, ax = plt.subplots(figsize=(8.5, 11))
fig.suptitle('Sample Size by Control Filters', fontweight="bold", y=1, **csfont)

cr = {
    'walk_size': 'red',
    'drive_size': 'black'}

df_sorted[df_sorted.index != '1500|all|all'][['walk_size','drive_size']].rename(columns={'walk_size':"Walking",'drive_size':"Driving"}).plot(kind='barh',stacked=True,ax=ax, color=['red','black']) 


for i, (index, row) in enumerate(df_sorted[df_sorted.index != '900|all|all'].iterrows()):
    # Calculate the total length of the stacked bar
    total_length = row['walk_size'] + row['drive_size']
    
    # Create the label text in the format "{walk_perc} / {drive_perc}"
    label_text = f"{row['walk_perc']:.0f} / {row['drive_perc']:.0f} %"
    
    # Place the label at the end of the bar
    ax.text(
        x=total_length + 5000,  # Slightly offset to the right of the bar
        y=i,  # Corresponds to the bar index
        s=label_text,
        va='center',
        ha='left',
        color='black',
        fontsize=9
    )
ax.set_yticklabels(
    df_sorted[df_sorted.index != '1500|all|all'].index,  # Use the index of the DataFrame for labels
    fontdict=hfont
)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.grid(axis='x', linestyle='--', color='gray', alpha=0.5) 
ax.set_ylabel('')
ax.ticklabel_format(axis='x', style='plain', useOffset=False)  # No scientific notation
plt.xticks(rotation=35)
plt.tight_layout()
#plt.savefig(r"C:\Users\katul\OneDrive\Escritorio\dataset_balance.jpg")
plt.show()


In [58]:
for dft in ('300',ds_300),('600',ds_600),('900',ds_900),('1200',ds_1200),('1500',ds_1500):
    dft[1].to_parquet(r"G:\Mi unidad\walknet_datalake\extractions\extraction_{di}.parquet".format(di = dft[0]))

In [ ]:
df_sorted